In [ ]:
import torch.nn as nn
embedding_sum = nn.EmbeddingBag(10, 3, mode = 'sum')
input = torch.LongTensor([1,2,4,5,4,3,2,9])
offsets = torch.LongTensor([0,4])
embedding_sum(input, offsets)

NameError: ignored

In [ ]:
# input, output 이 정확히 어떤 형태로 들어가는지 모르니까 그냥 그대로 따라하기 급급함
# 근데 어떤 형태로, 어떤식으로 변환해서 들어가느지 파악하기가 너무 어려움
# 전처리 함수와 클래스가 많고, 그 순서에 따라 추상적인 생각을 잘 하지 못함 

# 정리 
# embedding bag 의 input 값이 어떻게 생겼는지 알아야 함
# embe

In [ ]:
train_data = 'you need to know how to code'
word_set = set(train_data.split())
vocab = {tkn : i + 2 for i , tkn in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1

In [ ]:
import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings=len(vocab),  # 임베딩 해야할 단어 갯수, unique 단어집합
                               embedding_dim = 3, # 임베딩 할 벡터의 차원 , 직접 정할 수 있음
                               padding_idx = 1) # 패딩을 위한 토큰 인덱스

In [ ]:
t_data = df_raw[3][0]
word_set = set(t_data.split())
vocab = {w : i +2 for i, w in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1

In [ ]:
embedding_layer = nn.Embedding(num_embeddings = len(vocab),
                               embedding_dim = 512,
                               padding_idx = 10)

In [ ]:
embedding_sum = nn.EmbeddingBag(9, 5,mode = 'sum')
input = torch.LongTensor([1,3,5,7,10,1,3,5,7,10])
offsets = torch.LongTensor([0,3])

embedding_sum(input, offsets)

In [ ]:
import pandas as pd
import torch
import torchtext
from torchtext.datasets import text_classification
NGRAMS = 2
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
from torchtext.utils import download_from_url, extract_archive, unicode_csv_reader
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import tqdm

In [ ]:
df = pd.read_csv('/content/drive/My Drive/korean_embedding/raw_data/playstore_craw_data.csv', header = None)

In [ ]:
def ngrams_iterator(token_list, ngrams):
    """Return an iterator that yields the given tokens and their ngrams.
    Arguments:
        token_list: A list of tokens
        ngrams: the number of ngrams.
    Examples:
        >>> token_list = ['here', 'we', 'are']
        >>> list(ngrams_iterator(token_list, 2))
        >>> ['here', 'here we', 'we', 'we are', 'are']
    """

    def _get_ngrams(n):
        return zip(*[token_list[i:] for i in range(n)])

    #for x in token_list:
    #    yield x
    for n in range(2, ngrams + 1):
        for x in _get_ngrams(n):
            yield ' '.join(x)


In [ ]:
token_list = []
for row in df_raw.values:
    tokens = ' '.join(row[1:])
    tokens  = tokens.split(' ')
    token_list.extend(tokens)

index2token = { i : x for i, x in enumerate(set(token_list))}
token2index = { x : i for i, x in enumerate(set(token_list))}

label2cate = {}
cate2label = {}
for i, cate in enumerate(df_raw[5].unique()):
    label2cate[i] = cate 
    cate2label[cate] = i


In [ ]:
def _csv_iterator(df_raw, ngrams,  yield_cls=False):
    for row in df_raw.values:   
        tokens =' '.join(row[1:])
        tokens = tokens.split(' ')
        #token_idx = torch.tensor([token2index[t] for t in tokens])
    # print(tokens)
        if yield_cls :
            yield int(cate2label[row[0]]) - 1, ngrams_iterator(tokens, ngrams)
            #print(list(result))
        else :
            yield ngrams_iterator(tokens, ngrams)

In [ ]:
def _create_data_from_iterator(vocab, iterator, include_unk):
    data = []
    labels = []
    with tqdm.notebook.tqdm(unit_scale=0, unit='lines') as t:
        for cls, tokens in iterator:
            if include_unk:
                tokens = torch.tensor([vocab[token] for token in tokens])
            else:
                token_ids = list(filter(lambda x: x is not Vocab.UNK, [vocab[token]
                                        for token in tokens]))
                tokens = torch.tensor(token_ids)
            if len(tokens) == 0:
                logging.info('Row contains no tokens.')
            data.append((cls, tokens))
            labels.append(cls)
            t.update(1)
    return data, set(labels)

In [ ]:
include_unk = True
train_data, train_labels = _create_data_from_iterator(
        vocab, _csv_iterator(df_raw, ngrams, yield_cls=True), include_unk)

In [ ]:
train_data.get_vocab()

AttributeError: ignored

In [ ]:
1df_raw = df[[5,1,3]]

In [ ]:
label2cate = {}
cate2label = {}
for i, cate in enumerate(df_raw[5].unique()):
    label2cate[i] = cate 
    cate2label[cate] = i


In [ ]:
tokenizer = get_tokenizer("basic_english")

In [ ]:
tok=tokenizer('hi my name is jsp what is your name')
ngrams_iterator(tok, 2)

<generator object ngrams_iterator at 0x7fe027adf0f8>

In [ ]:
for row in df_raw[3]:
    tokens = ' '.join(row)
    print(len(tokens))
    break

2063


In [ ]:
vocab = build_vocab_from_iterator(ngrams_iterator(tokens, 2))

4121lines [00:00, 206720.49lines/s]


In [ ]:
vocab

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
TextSentiment()

NameError: ignored